In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import requests, re
from bs4 import BeautifulSoup
import pandas as pd
import unidecode

from tqdm import tqdm_notebook

In [5]:
def get_AOTY():

    df = pd.DataFrame(columns =['artist', 'album', 'rating'])

    for page in tqdm_notebook(range(1, 120+1)):
        url = "https://www.albumoftheyear.org/releases/" + str(page) + "/"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        tmptag = soup.select("#centerContent > div.flexContainer > div.wideLeft .albumBlock")

        for i in range(len(tmptag)):
            source = tmptag[i]

            artist = source.select(".artistTitle")[0].get_text()
            album = source.select(".albumTitle")[0].get_text()

            try:
                rating = source.select(".ratingRow .ratingBlock .rating")[0].get_text()
            except IndexError:
                rating = 0

            data = {
                'artist' : artist,
                'album' : album,
                'rating' : rating,
            }

            df.loc[len(df)] = data
    
    return df

In [6]:
rating_AOTY = get_AOTY()

HBox(children=(IntProgress(value=0, max=120), HTML(value='')))

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [7]:
print(rating_AOTY.shape)
rating_AOTY.head()

(8946, 3)


,artist,album,rating
0,Janelle Monáe,Dirty Computer,86
1,Post Malone,Beerbongs & Bentleys,53
2,Grouper,Grid of Points,79
3,Speedy Ortiz,Twerp Verse,76
4,Anne-Marie,Speak Your Mind,55


### replace accent characters and commas

In [8]:
def decodeuni(str):
    return unidecode.unidecode(str)

In [9]:
rating_AOTY['artist'] = rating_AOTY['artist'].apply(decodeuni)
rating_AOTY['album'] = rating_AOTY['album'].apply(decodeuni)
rating_AOTY['artist'] = rating_AOTY['artist'].apply(lambda x : x.replace(",", ""))
rating_AOTY['album'] = rating_AOTY['album'].apply(lambda x : x.replace(",", ""))

In [11]:
rating_AOTY.to_csv("../data/rating_AOTY.csv", index=False)

### Load csv file

In [12]:
df = pd.read_csv("../data/rating_AOTY_utf8.csv")

In [13]:
print(df.shape)
df.head()

(8946, 3)


,artist,album,rating
0,Janelle Monae,Dirty Computer,86
1,Post Malone,Beerbongs & Bentleys,53
2,Grouper,Grid of Points,79
3,Speedy Ortiz,Twerp Verse,76
4,Anne-Marie,Speak Your Mind,55


In [14]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [15]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [16]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [17]:
df.to_sql(name="rating_AOTY", con=engine, if_exists='replace')